# Projet d'optimisation 
## Par  Rania Fathi et Pierre Sion 

## I- Modélisation 

### Question 1 :
On définit l'élasticité-prix comme étant la variation relative de la quantité demandée par rapport au prix, soit encore: 
    $\varepsilon_i = \frac{\frac{dx_i}{x_i}}{\frac{dp_i}{p_i}} $ 
où l'on a noté $x_i$ la demande et $p_i$ le prix du ième produit 
    
Cela peut encore s'écrire: 
$\varepsilon_i = \frac{\partial x_i}{\partial p_i} \frac{p_i}{x_i}$

Comme on s'attend à ce que la demande pour un produit diminue si le prix augmente, on a $\varepsilon_i \leq 0$

Pour ce qui est de l'élasticité prix croisée entre le gouda et l'edam, elle vaut: 

$\varepsilon_c = \frac{\frac{dx_g}{x_g}}{\frac{dp_e}{p_e}} = \frac{\partial x_g}{\partial p_e} \frac{p_e}{x_g}$

Les biens considérés étant substituables, on a $\varepsilon_c \geq 0$ 

### Question 2 :
Le lait brut possède une quantité de matière grasse totale répartie entre les quatre produits, que l'on note $M$. En notant $m_i$ la teneur en matière grasse présente dans chaque produit, on a : 

$ \sum_{i=1}^{4}m_i x_i \leq M$

On fait par ailleurs le choix de mettre une inégalité plutôt qu'une égalité pour considérer le cas où tout le lait n'est pas transformé 

### Question 3 : 
De la même manière, on note $L$ la quantité totale de lactose contenue dans le lait, et $l_i$ celle contenue dans chaque produit, et on a alors la contrainte suivante: 

$\sum_{i=1}^{4} l_i x_i \leq L $

### Question 4 :

Afin de garantir la paix sociale, la moyenne (pondérée par la part de chaque produit dans le budget) des changements de prix relatifs ne doit pas être positive.
On note $b_i$ la part du ième produit dans le budget (déterminée à partir des données de l'année N-1). Cette contrainte s'écrit donc: 
$\sum_{i=1}^{4}b_i \frac{dp_i}{p_i} \leq 0$

Comme les contraintes dont on dispose jusque maintenant portent sur les quantités et non pas les prix, on la réécrit sous la forme suivante : $ \sum_{i=1}^4 \frac{b_i}{\epsilon_i} \frac{dx_i}{x_i} \leq 0$ 

### Question 5 :

_En supposant qu'on dispose déjà des $(p_i)_{1\leq i\leq4}$, $(b_i)_{1\leq i\leq4}$, $(\epsilon_i)_{1\leq i\leq4}$, $(m_i)_{1\leq i \leq 4}$, de $M$ et $L$_  
On dispose donc du problème d'optimisation sous contraintes suivant : $\max_{c(x) \leq 0} \Pi (x)$ 

où le profit $\Pi:\mathbf{R}^4 \rightarrow \mathbf{R}  $ vaut $\Pi(x) = \sum_{i=1}^4 p_i x_i $ (En toute rigueur, $\Pi$ est égal au revenu plutôt qu'au profit, mais l'omission du terme $-C$ correspondant au coût total du lait acheté ne modifie pas le problème d'optimisation à résoudre).

Et les contraintes $c : \mathbf{R}^4 \rightarrow \mathbf{R}^7$ sont les suivantes : 

$c_1(x) = - x_1 \\ 
c_2(x) = - x_2\\ 
c_3(x) = - x_3\\ 
c_4(x) = - x_4$

Ces quatre premières contraintes traduisent le fait que les quantités considérées sont nécessairement positives. Et nous disposons des deux contraintes des questions 2 et 3: 

$c_5(x) = \sum_{i=1}^{4}m_i x_i - M\\ 
c_6(x) = \sum_{i=1}^{4} l_i x_i - L $

Et enfin :
$c_7(x) =  \sum_{i=1}^4 \frac{b_i}{\epsilon_i} \frac{dx_i}{x_i} \leq 0$

NB: Il aurait été possible d'ajouter les contraintes portant sur le signe de l'élasticité des produits, imposant que l'augmentation du prix d'un produit réduit la demande, ou encore celles stipulant que les prix des produits se doivent d'être positifs 

## II- Implémentation 